In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("df_200_con_huecos.csv")
# Asegurar que la columna de fecha esté en formato datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Ver cuántos valores faltan por río
faltantes = df.groupby("rio_id")["altura_value"].apply(lambda s: s.isna().sum())
print("Valores faltantes por río:")
print(faltantes)


Valores faltantes por río:
rio_id
10-10         14
11-11         14
12-12         14
13-13         14
14-14         14
            ... 
9412-2837    148
9494-2844    190
9542-2813     14
9549-2809     54
99-99         14
Name: altura_value, Length: 189, dtype: int64


In [3]:
# Paso 1: Interpolación spline por cada río
df["altura_value"] = (
    df.groupby("rio_id")["altura_value"]
    .apply(lambda s: s.interpolate(method='spline', order=3))
)

# Verificamos los valores que siguen faltando
faltantes_despues_1 = df.groupby("rio_id")["altura_value"].apply(lambda s: s.isna().sum())
print("Faltantes después de la interpolación spline:")
print(faltantes_despues_1)


TypeError: incompatible index of inserted column with frame index

In [ ]:
# Paso 2: Relleno con promedio móvil de 7 días
df["altura_value"] = (
    df.groupby("rio_id")["altura_value"]
    .apply(lambda s: s.fillna(s.rolling(window=7, min_periods=1, center=True).mean()))
)

# Verificamos los faltantes restantes
faltantes_despues_2 = df.groupby("rio_id")["altura_value"].apply(lambda s: s.isna().sum())
print("Faltantes después del promedio móvil:")
print(faltantes_despues_2)


In [ ]:
# Agregamos una columna con el día del año
df["day_of_year"] = df["date"].dt.dayofyear

# Paso 3: Relleno con media estacional (por día del año)
df["altura_value"] = (
    df.groupby(["rio_id", "day_of_year"])["altura_value"]
    .transform(lambda s: s.fillna(s.mean()))
)

# Revisión final de faltantes
faltantes_final = df.groupby("rio_id")["altura_value"].apply(lambda s: s.isna().sum())
print("Faltantes después de aplicar los tres pasos:")
print(faltantes_final)


In [ ]:
# Elegí un río para visualizar
rio_ejemplo = df["rio_id"].unique()[0]
df_rio = df[df["rio_id"] == rio_ejemplo].copy()

# Para comparar, creamos una versión con los datos originales (sin imputar)
# (suponiendo que guardaste una copia antes de los reemplazos)
# Si no lo hiciste, ignorá esta parte y solo graficamos el resultado actual
try:
    df_original_rio = df_original[df_original["rio_id"] == rio_ejemplo]
    plt.plot(df_original_rio["date"], df_original_rio["altura_value"], 'o', alpha=0.4, label="Original (con huecos)")
except:
    print("⚠️ No se encontró df_original, mostrando solo los datos actuales imputados.")

# Graficamos la serie imputada
plt.plot(df_rio["date"], df_rio["altura_value"], '-', label="Imputada", linewidth=2)

plt.title(f"Evolución de altura del río {rio_ejemplo}")
plt.xlabel("Fecha")
plt.ylabel("Altura (m)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
